In [1]:
'''
Data will be extracted from https://sunrise-sunset.org/ Restful API. It returns multiple key positions of 
the sun based on a given location and date, all time data is returned in UTC time.
'''

'\nData will be extracted from https://sunrise-sunset.org/ Restful API. It returns multiple key positions of \nthe sun based on a given location and date, all time data is returned in UTC time.\n'

In [2]:
import pandas as pd
import requests
import json
import time
from pandas.io.json import json_normalize
from datetime import date,datetime, timedelta

In [3]:
# load cities location from csv file

cities_locations = pd.read_csv('cities_lat_lng.csv') 
cities_locations.head()

,city,country,lat,lng
0,Tokyo,Japan,35.4,139.45


In [58]:
class Sunrise_sunset_data_extractor():
    """
        This class knows how to obtain data from sunrise_sunset_org and returns a json object with the loaded data.
        It requires a csv input file with data of the cities locations you want to get data from.
        
        use get_sunrise_sunset_json() method to get latest obtained data.
        run construct_sunrise_sunset_json() to update load data.
        
        format of output object: 
        {'date':
            'sunrise_sunset_data':[
                {'location':{'country': ,'city': ,'lat': ,'lng': }
                'sunrise_sunset_times':{}},

                {'location':{'country': ,'city': ,'lat': ,'lng': }
                'sunrise_sunset_times':{}}]}
    """
    def __init__(self, location_csv_path, dates_range):
        self.location_csv_path = location_csv_path
        self.dates_range = dates_range
        self.locations_df = self.get_locations_df()
     #   self.construct_sunrise_sunset_json()
    
    def get_locations_df(self):
        # returns load data of locations csv file in df.
        return pd.read_csv(self.location_csv_path)

    def get_sunrise_sunset_json(self):
        return self.sunrise_sunset_json
    
    def construct_sunrise_sunset_json(self):
        self.sunrise_sunset_json = {}
        # iterates through the given dates, obtaining for each the set of locations from the locations csv file.
        for date in self.dates_range:
            all_loc_times_data_for_date = []
            for index, row in sunrise_sunset_data_extractor.locations_df.iterrows():
                
                # we add a delay time to not stress restful api from sunrise-sunset-org. 10 seconds
                time.sleep(10)
                print("debug: starting request to sunrise-sunset-org restful api.")

                loc_data_obj = self.get_location_obj(row)
                sunrise_sunset_data_obj = sunrise_sunset_data_extractor.get_sunrise_sunset_data(row, date)
                loc_times_data_obj = {'location': loc_data_obj, 'sunrise_sunset_times': sunrise_sunset_data_obj['results']}
                all_loc_times_data_for_date.append(loc_times_data_obj)
            self.sunrise_sunset_json[date.strftime('%Y-%m-%d')] = all_loc_times_data_for_date
            
        
    def get_sunrise_sunset_data(self, city_location_row, date):
        # define data extraction method
        try:
            date_str = date.strftime('%Y-%m-%d')
            url_template = 'https://api.sunrise-sunset.org/json?lat={lat}&lng={lng}&date={date}'

            url = url_template.format(
                lat = city_location_row[2],
                lng = city_location_row[3],
                date = date_str
            )


            response = (requests.get(url).text)
            response_json = json.loads(response)
            time.sleep(0.5)
            return response_json

        except Exception as e:
            raise e    
    
    def get_location_obj(self, city_location_row):
        obj = {}
        obj['city'] = city_location_row[0]
        obj['country'] =  city_location_row[1]
        obj['lat'] =  city_location_row[2]
        obj['lng'] = city_location_row[3]

        return obj
        

In [50]:
def create_dates_range(start_date, end_date, days_interval):
    """
        Returns a range of dates from the given start_date to the end_date, increasing by a constant day interval.
        
        usage example:
        start_date= datetime.strptime('2018-01-01', '%Y-%m-%d')
        end_date= datetime.strptime('2018-12-31', '%Y-%m-%d')

        create_dates_range(start_date, end_date, 14)
    """
    if (start_date > end_date):
        raise Exception("Cannot create date range because start_date > end_date.")  
    
    add_date =  start_date
    datelist = []
    
    while (add_date < end_date):
        datelist.append(add_date)
        add_date = add_date + timedelta(days= days_interval)
        
    return datelist


In [51]:
start_date= datetime.strptime('2018-01-01', '%Y-%m-%d')
end_date= datetime.strptime('2018-01-15', '%Y-%m-%d')

dates_range = create_dates_range(start_date, end_date, 14)

sunrise_sunset_data_extractor = Sunrise_sunset_data_extractor('cities_lat_lng.csv',dates_range)

In [52]:
sunrise_sunset_data_extractor.construct_sunrise_sunset_json()

debug: starting request to sunrise-sunset-org restful api.


In [54]:
sunrise_sunset_data_extractor.get_sunrise_sunset_json()

{'2018-01-01': [{'location': {'country': 'Tokyo',
    'city': ' Japan',
    'lat': 35.4,
    'lng': 139.45},
   'sunrise_sunset_times': {'sunrise': '9:51:10 PM',
    'sunset': '7:40:27 AM',
    'solar_noon': '2:45:48 AM',
    'day_length': '09:49:17',
    'civil_twilight_begin': '9:23:05 PM',
    'civil_twilight_end': '8:08:32 AM',
    'nautical_twilight_begin': '8:51:21 PM',
    'nautical_twilight_end': '8:40:16 AM',
    'astronomical_twilight_begin': '8:20:25 PM',
    'astronomical_twilight_end': '9:11:11 AM'}}]}

In [57]:
with open('sunrise_sunset_times_data.json', 'w') as outfile:  
    json.dump(sunrise_sunset_data_extractor.get_sunrise_sunset_json(), outfile)